<a href="https://colab.research.google.com/github/WilliamAshbee/DataminingAssignments/blob/main/DM_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn as sk
import pandas as pd
import numpy as np


In [2]:
#!wget http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleve.mod

In [3]:
#url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleve.mod'
#df1 = pd.read_csv(url,header=None,skiprows=(i for i in range(20)),delim_whitespace=True,error_bad_lines=False)



In [4]:
pain = {'angina': 1, 'asympt': 4,'notang':2,'abnang':3}
gender = {'male':1,'fem':0}
bsl20 = {'true':1,'fal':0}
restingecg = {'norm':0,'abn':1, 'hyp':2}
exindang = {'true':1,'fal':0}
slope = {'up':1, 'flat':2, 'down':3}
thal = {'norm':1, 'fix':2, 'rev':3}
state= {'buff':1,'sick':-1}
code = {'H':0,'S1':1,'S2':2,'S3':3,'S4':4}
eleventh = {'?':-10000}
twelth = {'?':-10000}
#df2 = df1.replace({1: gender,2:pain,5:bsl20,6:restingecg,8:exindang,10:slope,11:eleventh,12:{**thal, **twelth},13:state,14:code})


In [5]:
#https://stackoverflow.com/questions/17114904/python-pandas-replacing-strings-in-dataframe-with-numbers
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html

In [6]:
#df2.tail()

In [7]:

#df2n = df2.to_numpy().astype(float)

#df2n.shape


In [8]:
url2 = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
df_clev = pd.read_csv(url2,header=None,error_bad_lines=False)

In [9]:
questionmark = {'?':'-10000'}

dfcn = df_clev
dfcn = dfcn.replace({11:questionmark,12:questionmark})
dfcn = dfcn.to_numpy().astype(float)

print(np.sum(dfcn=='?'))#,np.sum(df2n=='?'))

0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


In [70]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

import os
#change here
y = dfcn[:,13]

X = dfcn[:,:13]
#min_max_scaler = preprocessing.MinMaxScaler()
#X = min_max_scaler.fit_transform(X)
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.fit_transform(X)

LR = LogisticRegression(max_iter = 10000,random_state=0, solver='lbfgs', multi_class='auto').fit(X, y)
round(LR.score(X,y), 5)


0.60726

In [71]:
out = LR.predict(X)

In [72]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo')
print(clf.kernel)


rbf


In [73]:
C_2d_range = [1e-2, 1, 1e2]
gamma_2d_range = [1e-1, 1, 1e1]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        clf = svm.SVC(C=C, gamma=gamma,decision_function_shape='ovo')
        clf.fit(X, y)
        out = clf.predict(X)
        if np.mean(y==out) > .6:
          print(np.mean(y==out),C,gamma)
        classifiers.append((C, gamma, clf,np.mean(y==out)))

clf = svm.SVC(C=1, gamma=.1,decision_function_shape='ovo')
clf.fit(X, y)
clf.predict(X)

0.7194719471947195 1 0.1
0.976897689768977 1 1
1.0 1 10.0
1.0 100.0 0.1
1.0 100.0 1
1.0 100.0 10.0


array([0., 1., 1., 0., 0., 0., 3., 0., 0., 1., 0., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 3., 0., 2., 0., 0.,
       0., 0., 3., 1., 2., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 3., 0., 0., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.,
       2., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 3., 0.,
       0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 1., 3., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 2., 0., 1., 2., 0., 2., 0., 0., 0., 0., 3.,
       2., 3., 0., 0., 3., 0., 0., 3., 3., 0., 0., 0., 0., 0., 0., 0., 0.,
       3., 0., 3., 0., 0., 0., 0., 1., 0., 0., 3., 0., 0., 0., 0., 0., 0.,
       1., 3., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       2., 1., 1., 0., 1., 1., 0., 3., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 3., 1., 2., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 3., 1., 0.

In [74]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=30000,learning_rate_init = .00001).fit(X, y)
clf.predict(X)
clf.score(X, y)


0.6039603960396039

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#X, y = make_classification(n_samples=1000, n_features=4,
#...                            n_informative=2, n_redundant=0,
#...                            random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(X, y)
print(clf.score(X,y))


0.9702970297029703


In [76]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X, y)
gnb_score = gnb.score(X,y)
print(gnb_score)

0.1848184818481848


In [77]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X, y)
print(neigh.score(X,y))

0.693069306930693


In [78]:
#https://medium.com/@rrfd/boosting-bagging-and-stacking-ensemble-methods-with-sklearn-and-mlens-a455c0c982de

In [79]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

clf = KNeighborsClassifier()
clf.fit(X,y)
bagging_clf = BaggingClassifier(clf, max_samples=0.4, max_features=4, random_state=0)
bagging_scores = cross_val_score(bagging_clf, X, y, cv=10, n_jobs=-1)
print(bagging_scores)
knnscores = cross_val_score(clf, X, y, cv=10, n_jobs=-1)
print(knnscores)

[0.5483871  0.58064516 0.5483871  0.56666667 0.53333333 0.6
 0.6        0.56666667 0.5        0.5       ]
[0.64516129 0.58064516 0.61290323 0.46666667 0.56666667 0.6
 0.6        0.63333333 0.46666667 0.43333333]


In [80]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(base_estimator=None)
clf.fit(X,y)
clf.score(X,y)
cross_val_score(clf, X, y, cv=10, n_jobs=-1)

array([0.67741935, 0.58064516, 0.5483871 , 0.66666667, 0.53333333,
       0.46666667, 0.5       , 0.43333333, 0.46666667, 0.46666667])

In [81]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
grad_boost = GradientBoostingClassifier()
cross_val_score(grad_boost, X, y, cv=10, n_jobs=-1)

array([0.64516129, 0.51612903, 0.61290323, 0.56666667, 0.6       ,
       0.43333333, 0.46666667, 0.66666667, 0.53333333, 0.53333333])

In [82]:
#
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import RidgeClassifier
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = svm.SVC()
rg = RidgeClassifier()

clf = [rf, et, knn, svc, rg]
eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')
for clf, label in zip([rf, et, knn, svc, rg, eclf], ['Random Forest', 'Extra Trees', 'KNeighbors', 'SVC', 'Ridge Classifier', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


Accuracy: 0.56 (+/- 0.06) [Random Forest]
Accuracy: 0.57 (+/- 0.07) [Extra Trees]
Accuracy: 0.56 (+/- 0.07) [KNeighbors]
Accuracy: 0.55 (+/- 0.03) [SVC]
Accuracy: 0.56 (+/- 0.03) [Ridge Classifier]
Accuracy: 0.57 (+/- 0.04) [Ensemble]


In [97]:
from sklearn.metrics import f1_score
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]
f1_score(y_true, y_pred, average=None)



array([0.8, 0. , 0. ])

In [84]:
from sklearn import metrics
scores = cross_val_score(
clf, X, y, cv=5, scoring='f1_macro')


In [85]:
scores

array([0.28080808, 0.31384615, 0.20992785, 0.205     , 0.248     ])

In [86]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html

In [ ]:
#note multiclass roc curve is a little poorly defined, so i went with
#solution from: https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
#runtime on google colab was about 4 minutes
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score


cv = StratifiedKFold(n_splits=10)
y = y.squeeze()

#code multiclass_roc_auc_score from medium article above and below
#https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
  lb = preprocessing.LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return roc_auc_score(y_test, y_pred, average=average)

import random
def getMonteCarloSearchParams():#random search is faster than grid search for most purposes
  C = random.choice(np.logspace(-6, -1, 10).tolist())
  gamma = random.choice([0.001, 0.01, 0.1, 1])
  k = random.choice([3,4,5,6,7])
  d = random.choice([3,4,5,6,7])
  maxit = random.choice([10**3,10**4,10**5,10**6])

  return C,gamma,k,d,maxit

epochs = 10

scores = np.zeros((epochs,10))
aucs = np.zeros((epochs,10))
f1_scores = np.zeros((epochs,10))

params = np.zeros((epochs,5))


for e in range(epochs):
  C,gamma,k,d,maxit = getMonteCarloSearchParams()
  params[e,0] = C
  params[e,1] = gamma
  params[e,2] = k
  params[e,3] = d
  params[e,4] = maxit
  
  print('\nc',C,'gamma',gamma,'k',k,'d',d,'maxit',maxit)
  rfc = RandomForestClassifier(max_depth=d)
  knnc = KNeighborsClassifier(n_neighbors=k)
  bagging_knnc = BaggingClassifier(knnc, max_samples=0.4, max_features=4)
  lrc = LogisticRegression(max_iter = maxit, solver='lbfgs', multi_class='auto')
  svc = svm.SVC(kernel = 'rbf',gamma = gamma,max_iter = maxit,C= C,decision_function_shape='ovo', probability=True)
  adac = AdaBoostClassifier(base_estimator=svc)
  vc = VotingClassifier(estimators=[("logistic regression",lrc),('Random Forests', rfc), ('KNeighbors', bagging_knnc), ('adaSVCrbf', adac)], voting='hard')
  classifier = vc
  
  
  for i, (train, test) in enumerate(cv.split(X, y)):
    print('\nepoch',e,' cv of 10:',i+1)
    classifier.fit(X[train], y[train])
    y_pred = classifier.predict(X[test])

    scores[e,i] = classifier.score(X[test],y[test])
    aucs[e,i] = multiclass_roc_auc_score(y[test],y_pred)
    f1_scores[e,i] = f1_score(y[test], y_pred, average='macro')
    print("score",'{:.2f}'.format(scores[e,i]))
    print('auc score','{:.2f}'.format(aucs[e,i]))
    print('f1 score', '{:.2f}'.format(f1_scores[e,i]))



In [145]:
#"C,gamma,k,d,maxit"
sc_par = params[np.argmax(np.mean(scores,axis=1)),:]
auc_par = params[np.argmax(np.mean(aucs,axis=1)),:]
f1_par = params[np.argmax(np.mean(f1_scores,axis=1)),:]

 

print('best average scores cross validation value:',"{:.2f}".format(np.max(np.mean(scores,axis =1 ))))
print('model params C,gamma,k,d,maxit:','{:e} {:6g} {:d} {:d} {:d} '.format(sc_par[0],sc_par[1],(int)(sc_par[2]),(int)(sc_par[3]),(int)(sc_par[4])))

print('\nbest average auc value:',"{:.2f}".format(np.max(np.mean(aucs,axis =1 ))))
print('model params C,gamma,k,d,maxit:','{:e} {:6g} {:d} {:d} {:d} '.format(auc_par[0],auc_par[1],(int)(auc_par[2]),(int)(auc_par[3]),(int)(auc_par[4])))

print('\nbest average f1score value:',"{:.2f}".format(np.max(np.mean(f1_scores,axis =1 ))))
print('model params C,gamma,k,d,maxit:','{:e} {:6g} {:d} {:d} {:d} '.format(f1_par[0],f1_par[1],(int)(f1_par[2]),(int)(f1_par[3]),(int)(f1_par[4])))


best average scores cross validation value: 0.57
model params C,gamma,k,d,maxit: 2.154435e-03    0.1 4 4 1000 

best average auc value: 0.54
model params C,gamma,k,d,maxit: 2.154435e-03    0.1 4 4 1000 

best average f1score value: 0.23
model params C,gamma,k,d,maxit: 2.154435e-03    0.1 4 4 1000 


In [ ]:
#https://www.kaggle.com/iamhungundji/covid19-symptoms-checker
#bonus work dataset: predict severity